In [102]:
import glob
import os
import pandas as pd
import sklearn
import sklearn.metrics

# folder = "/Users/johnmorris/Projects/interpretable-embeddings/scripts/d3/out"
folder = 'out'
paths = glob.glob(os.path.join(folder, "d3_*.csv"))

paths[0]

'out/d3_34_bad.csv'

In [103]:
def read_csv(path):
    lines = open(path).readlines()
    lines = lines[7:]  # discard header
    all_data = []
    for line in lines:
        idx, data_str = eval(line)
        all_data.append(eval(data_str))
    return pd.DataFrame(all_data, columns=['task', 'idx', 'true_label', 'pred_label', 'answer', 'model'])


dfs = [read_csv(path) for path in paths]
df = pd.concat(dfs)

df['correct'] = df['true_label'] == df['pred_label']

# add example_len
df['example_len'] = None
for task in df.task.unique():
    task_str = '_'.join(task.split('_')[:-1])
    examples = pd.read_csv(f'd3_processed/{task_str}.csv')
    examples['example_len'] = examples['input'].apply(lambda x: len(x.split()))
    # set example_len for each task based on "idx" column
    df.loc[df.task == task, 'example_len'] = df[df.task ==
                                                task].idx.apply(lambda x: examples.loc[x].example_len)

In [104]:
df.head()

,task,idx,true_label,pred_label,answer,model,correct,example_len
0,d3_34_bad,228,0,0,no,openai,True,79
1,d3_34_bad,228,0,0,no,meta,True,79
2,d3_34_bad,228,0,0,no,mistral,True,79
3,d3_34_bad,8,1,1,yes,openai,True,275
4,d3_34_bad,8,1,1,yes,meta,True,275


In [123]:
df.groupby('model')['correct'].mean()
# df[df.example_len <= 100000].groupby('model')['correct'].mean()

model
meta       0.765487
mistral    0.763563
openai     0.737207
Name: correct, dtype: float64

In [106]:
df.groupby(['model', 'task'])['correct'].mean()

model   task                
meta    d3_0_irony              0.58
        d3_10_infrastructure    0.82
        d3_12_medical           0.85
        d3_13_water             0.87
        d3_14_search            0.85
                                ... 
openai  d3_5_evacuate           0.83
        d3_6_terorrism          0.95
        d3_7_crime              0.83
        d3_8_shelter            0.84
        d3_9_food               0.87
Name: correct, Length: 156, dtype: float64

In [122]:
df.groupby(['model'])['correct'].sem()

model
meta       0.005877
mistral    0.005894
openai     0.006106
Name: correct, dtype: float64

In [96]:
for model in df['model'].unique():
    model_df = df[df['model'] == model]
    print(model, sklearn.metrics.f1_score(
        model_df['true_label'], model_df['pred_label']))

openai 0.6484817292846115
meta 0.6375260184359203
mistral 0.6499572771290231


In [97]:
# .reset_index().pivot(index='task', columns='model', values='correct').to_latex()
d = df.groupby(['model', 'task'])['correct'].mean().reset_index()

In [98]:
d['correct'] = d['correct'].apply(lambda x: max(x, 1 - x))

In [101]:
d.groupby('model')['correct'].mean()

model
meta       0.781453
mistral    0.787648
openai     0.773607
Name: correct, dtype: float64

In [100]:
# fullly display d
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(d)

,model,task,correct
0,meta,d3_0_irony,0.580000
1,meta,d3_10_infrastructure,0.820000
2,meta,d3_12_medical,0.850000
3,meta,d3_13_water,0.870000
4,meta,d3_14_search,0.850000
5,meta,d3_15_utility,0.640000
6,meta,d3_16_hillary,0.820000
7,meta,d3_17_hillary,0.860000
8,meta,d3_18_offensive,0.590000
9,meta,d3_19_offensive,0.530000
